In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!pip install monai
!pip install nibabel
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 14.5 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import nibabel as nib
from monai.transforms import Spacing
from monai.transforms import SpatialPad
from monai.transforms import Resize
from monai.transforms import CropForeground

import time
import torch

In [5]:
!pwd

/content


In [6]:
os.cpu_count()

2

# Load Image and Label

In [7]:
!ls /gdrive/MyDrive/LiTS_sample

archive.zip  image_128	 logs  mask_128   models  preds_128
image	     image_crop  mask  mask_crop  preds


## Resample all same size

In [8]:
!ls $image_data_dir

sample_data


In [22]:
from multiprocessing import Pool

class Resampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = (np_image>0.5).astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        np.save(os.path.join(target_dir, file_name), np_image)

    def resize(self, np_image): 
        if self.is_label:
            np_image = np_image > 1
        resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        # # load npy
        np_file = np.load(os.path.join(origin_dir, file), allow_pickle=True)
        # load nifti
        # np_file = nib.load(os.path.join(origin_dir, file)).get_fdata()
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=2)
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

class MultiChResampler:
    def __init__(self, origin_dir, target_dir, resample_size, is_label=False):
        self.origin_dir, self.target_dir, self.resample_size, self.is_label = origin_dir, target_dir, resample_size, is_label

    def save_np(self, np_image, target_dir, file_name):
        if self.is_label:
            np_image = np_image.astype(np.uint8)            
        else:
            np_image = np_image.astype(np.int16)
        
        # For torch order
        np_image = np.flip(np_image.transpose([2,1,0]), axis=(1,2))
        np.save(os.path.join(target_dir, file_name), np_image) 

    def resize(self, np_image): 
        if self.is_label:
            resizer = Resize(self.resample_size, mode='nearest')
        else: 
            resizer = Resize(self.resample_size)
        resized_image = resizer(np_image[np.newaxis,:]) # with channel
        return resized_image[0] # without channel

    def resample(self, file, origin_dir, target_dir, resample_func, save_func):
        # # load npy
        # np_file = np.load(os.path.join(origin_dir, file))
        # load nifti
        np_file = nib.load(os.path.join(origin_dir, file)).get_fdata()
        resampled_image = resample_func(np_file)
        save_func(resampled_image, target_dir, file)

    def wrapper(self, args):
        return self.resample(*args)

    def resample_all(self):
        os.makedirs(self.target_dir, exist_ok=True)

        t = time.perf_counter()

        file_list = os.listdir(self.origin_dir)
        arg_list = [(file, self.origin_dir, self.target_dir, self.resize, self.save_np) for file in file_list]
        pool = Pool(processes=os.cpu_count())
        pool.map(self.wrapper, arg_list)
        pool.close()
        pool.join()

        elapsed_time = time.perf_counter() - t

        print(f'''
        consumed_time: {elapsed_time}s
        ''')

In [18]:
source_dir = '/gdrive/MyDrive/LiTS_sample/image'
target_dir = '/gdrive/MyDrive/LiTS_sample/image_128'

source_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask'
target_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask_128'

target_shape = [128,128,128]

image_resampler = MultiChResampler(source_dir, target_dir, target_shape)
label_resampler_1 = MultiChResampler(source_dir_lb, target_dir_lb, target_shape, True)

In [ ]:
image_resampler.resample_all()


        consumed_time: 343.88992797100036s
        


In [ ]:
label_resampler_1.resample_all()


        consumed_time: 305.358029944s
        


In [ ]:
file_names = os.listdir(target_dir)
new_file_names = [file_name.split('.')[0] + '.npy' for file_name in file_names]

In [ ]:
for (old_file, new_file) in zip(file_names, new_file_names): 
  old_path = os.path.join(target_dir, old_file)
  new_path = os.path.join(target_dir, new_file)
  os.rename(old_path, new_path)

In [ ]:
print(len(os.listdir(target_dir)))
os.listdir(target_dir)[:3]

51


['volume-0.npy', 'volume-1.npy', 'volume-6.npy']

In [ ]:
os.listdir(target_dir_lb)[:3]

['segmentation-0.nii.npy', 'segmentation-1.nii.npy', 'segmentation-15.nii.npy']

In [ ]:
for i in range(51):
  old_file = f"segmentation-{i}.nii.npy"
  old_path = os.path.join(target_dir_lb, old_file)
  new_file = f"volume-{i}.npy"
  new_path = os.path.join(target_dir_lb, new_file)
  os.rename(old_path, new_path)

In [ ]:
file_names = os.listdir(target_dir)
image_sizes = [np.load(os.path.join(target_dir, file)).shape for file in file_names]
label_sizes = [np.load(os.path.join(target_dir_lb, file)).shape for file in file_names]

In [ ]:
label_sizes

[(128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128),
 (128, 128, 128)]

# Test Codes

In [10]:
180*180*96

3110400

In [27]:
source_dir = '/gdrive/MyDrive/LiTS_sample/image_crop'
target_dir = '/gdrive/MyDrive/LiTS_sample/image_crop_180'

source_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask_crop'
target_dir_lb = '/gdrive/MyDrive/LiTS_sample/mask_crop_180'

target_shape = [96,180,180]

image_resampler = Resampler(source_dir, target_dir, target_shape)
label_resampler = Resampler(source_dir_lb, target_dir_lb, target_shape, True)

In [28]:
image_resampler.resample_all()
label_resampler.resample_all()


        consumed_time: 13.80966657600004s
        

        consumed_time: 28.51845971099999s
        
